In [10]:
import xarray as xr
import matplotlib.pyplot as plt
from pyclim_noresm.general_util_funcs import global_avg
import pandas as pd
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

In [11]:
time_slice = snakemake.params.get('time_slice',slice(None,None))
vmin = snakemake.params.get('vmin',-2.5)
vmax = snakemake.params.get('vmax', 1)
start_year = snakemake.params.get('start_year',1950)

paths = sorted(snakemake.input.paths)
vname = snakemake.wildcards.vName
params = snakemake.params

In [12]:
dsets = {}
for path in paths:

    ds = xr.open_dataset(path)
    source_id = ds.source_id
    
    ds = ds.rename({'year':'time'})
    ds = ds[vname].sel(time=slice(start_year,None))
    ds = global_avg(ds)
    dsets[source_id] = ds.to_pandas()
    #time=ds.time.values

df=pd.DataFrame(dsets)
if start_year:
    df = df.loc[start_year:,:]


label = params.get('label', ds.long_name)


In [13]:
fig, ax = plt.subplots(figsize=(10,5))
ax.axhline(y=0, color='k', linewidth=3.1)
df.plot(ax=ax, alpha = 0.3, color=["#1845fb", "#ff5e02", "#c91f16", 
                                   "#c849a9", "#adad7d", "#86c8dd", "#578dff", "#656364"],
       legend=False)
for line in ax.lines: # put this before you call the 'mean' plot function.
    line.set_label(s='')
df.rolling(5,center=True).mean().plot(ax=ax, 
                          color=["#1845fb", "#ff5e02", "#c91f16", "#c849a9", 
                                 "#adad7d", "#86c8dd", "#578dff", "#656364"],
                         linewidth=2.1, legend=False)
ax.set_ylabel(f'$\Delta ${vname} histSST histSST-piAer \n W m-2', fontsize=14)

ax.set_ylim(vmin,vmax)
ax.xaxis.set_major_locator(MultipleLocator(5))
ax.set_xlabel('')
h,l = ax.get_legend_handles_labels()
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.legend(fontsize=14)
plt.savefig(snakemake.output.outpath, dpi=144, bbox_inches='tight')